In [34]:
import numpy as np
from copy import deepcopy
from scipy import ndimage

In [35]:
grid1 = [[np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
         [np.nan,np.nan,np.nan,1,np.nan,np.nan,np.nan,np.nan,np.nan,6,5,np.nan],
         [np.nan,np.nan,np.nan,np.nan,np.nan,3,np.nan,np.nan,np.nan,np.nan,6,np.nan],
         [np.nan,4,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,7,np.nan,np.nan,np.nan],
         [np.nan,np.nan,np.nan,np.nan,2,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,7],
         [np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan,np.nan,3,7,np.nan,np.nan],
         [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
         [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
         [np.nan,np.nan,np.nan,7,np.nan,5,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
         [np.nan,5,np.nan,np.nan,np.nan,7,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
         [np.nan,6,7,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan],
         [np.nan,np.nan,np.nan,np.nan,6,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]]
grid1 = np.matrix(grid1)
print(grid1)

topfirst1 =    [6,36,30,34,27,3,40,27,None,None,7,None]
bottomfirst1 = [6,None,None,4,None,None,None,None,None,None,None,5]
leftfirst1 =   [5,7,7,33,29,2,40,28,None,None,36,None]
rightfirst1 =  [4,None,None,1,None,None,None,None,None,None,None,7]

[[nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan  1. nan nan nan nan nan  6.  5. nan]
 [nan nan nan nan nan  3. nan nan nan nan  6. nan]
 [nan  4. nan nan nan nan nan nan  7. nan nan nan]
 [nan nan nan nan  2. nan nan nan nan nan nan  7.]
 [nan nan  6. nan nan nan nan nan  3.  7. nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan  7. nan  5. nan nan nan nan nan nan]
 [nan  5. nan nan nan  7. nan nan nan nan nan nan]
 [nan  6.  7. nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan  6. nan nan nan nan nan nan nan]]


In [36]:
def possible(yn,xn,n,originalgrid,topfirst,bottomfirst,leftfirst,rightfirst):    
        
    #Create a copy of the original grid
    grid = deepcopy(originalgrid)
    
    #Insert new val
    grid.itemset((yn, xn), n)
    
    #If there are no nan, check sum of row and col
    if not np.isnan(grid[yn]).any():
        if grid[yn].sum() != 20:
            return False
    

    if not np.isnan(grid[:,xn]).any():
        if grid[:,xn].sum() != 20:
            return False
        
    #Check if new val is allowed by top/bottom/left/right first
    if topfirst[xn] != None:
        for el in grid[:,xn]:
            if np.isnan(el) or el == topfirst[xn]:
                break
            elif el == 0:
                continue
            else:
                return False
            
    if bottomfirst[xn] != None:
        for el in reversed(grid[:,xn]):
            if np.isnan(el) or el == bottomfirst[xn]:
                break
            elif el == 0:
                continue
            else:
                return False
            
    if leftfirst[yn] != None:
        for el in grid[yn].T:
            if np.isnan(el) or el == leftfirst[yn]:
                break
            elif el == 0:
                continue
            else:
                return False
            
    if rightfirst[yn] != None:
        for el in reversed(grid[yn].T):
            if np.isnan(el) or el == rightfirst[yn]:
                break
            elif el == 0:
                continue
            else:
                return False
        
    #After the first checks for nan, replace nan with zeros
    grid[np.isnan(grid)] = 0
    grid = grid.astype(int)
    
    #Putting a zeros is almost always ok
    if n == 0:
        return True
    
    #Check if there are no more than n val equal to n
    if np.count_nonzero(grid == n) > n:
        return False
    
    #Check neighbouring squares
    grid = grid.tolist()
    try: #Top left square
        topleftcorner = grid[yn-1][xn-1]
        top = grid[yn-1][xn]
        left = grid[yn][xn-1]
        if topleftcorner != 0 and top != 0 and left != 0:
            return False
    except IndexError:
        pass
    
    try: #Top right square
        toprightcorner = grid[yn-1][xn+1]
        top = grid[yn-1][xn]
        right = grid[yn][xn+1]
        if toprightcorner != 0 and top != 0 and right != 0:
            return False
    except IndexError:
        pass
    
    try: #Bottom left square
        bottomleftcorner = grid[yn+1][xn-1]
        bottom = grid[yn+1][xn]
        left = grid[yn][xn-1]
        if bottomleftcorner != 0 and bottom != 0 and left != 0:
            return False
    except IndexError:
        pass
    
    try: #Bottom right square
        bottomrightcorner = grid[yn+1][xn+1]
        bottom = grid[yn+1][xn]
        left = grid[yn][xn+1]
        if bottomrightcorner != 0 and bottom != 0 and right != 0:
            return False
    except IndexError:
        pass 
        
    #If the number passes all selections return True
    return True    

In [37]:
def solve(grid,topfirst,bottomfirst,leftfirst,rightfirst):
    
    for y in range(7):
        for x in range(7):
            if np.isnan(grid.item((y,x))):
                for n in reversed(range(0,7)):
                    if possible(y,x,n,grid,topfirst,bottomfirst,leftfirst,rightfirst):
                        grid.itemset((y, x), n)
                        sol = solve(grid,topfirst,bottomfirst,leftfirst,rightfirst)
                        try:
                            if sol == None:
                                pass
                        except ValueError:
                            return sol  
                        grid.itemset((y, x), np.nan)
                return 
    
    #Check whether the grid is a fully-connected island
    _ , numberfeatures = ndimage.label(grid)
    if numberfeatures == 1:
        return grid

In [38]:
g1 = deepcopy(grid1)
sol1 = solve(g1,topfirst1,bottomfirst1,leftfirst1,rightfirst1)
print(sol1)

None


In [39]:
solution = sol1
print(solution)

None


In [40]:
tot = 0
for y in range(7):
    for x in range(7):
        tot += pow(solution.item(y,x),2)
print(tot)

AttributeError: 'NoneType' object has no attribute 'item'